> <p><small><small>This Notebook is made available subject to the licence and terms set out in the <a href = "http://www.github.com/google-deepmind/ai-foundations">AI Research Foundations Github README file</a>.

<img src="https://storage.googleapis.com/dm-educational/assets/ai_foundations/GDM-Labs-banner-image-C1-white-bg.png">

# Lab: Experiment with N-Gram Models

<a href='https://colab.research.google.com/github/google-deepmind/ai-foundations/blob/master/course_1/gdm_lab_1_2_experiment_with_n_gram_models.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

Practice extracting n-gram counts and learn how to use them to build a language model.


30 minutes

## Overview

In the previous lab, you built a very small language model in which you manually assigned probabilities to the next token for different prompts. In this lab, you will automatically estimate the probabilities for predicting the next word and build an **n-gram model** using a small dataset of paragraphs. This will result in a language model that will be able to predict the next word for a given prompt and that can be used to generate texts. You will also gain a practical understanding of how n-gram models capture language patterns and what the limitations of this family of models are. This knowledge will serve as a foundation for exploring more advanced language models in later modules.

### What you will learn:

By the end of this lab, you will be able to:

* Split paragraphs in a dataset into word-like units called tokens, a process known as **tokenization**.
* Estimate the probabilities for an n-gram language model from a dataset.
* Use the n-gram language model to predict individual tokens and longer continuations.



### Tasks

As mentioned in the previous article, an n-gram is a continuous sequence of $n$ words. An n-gram model uses these sequences to estimate the probability of the next word given a preceding sequence of $n-1$ words (the context).

Recall that you can compute the probability $P(\mbox{B} \mid \mbox{A})$, where $\mbox{B}$ is the next word and $\mbox{A}$ is the context, as follows:

$$P(\mbox{B} \mid \mbox{A}) = \frac{\mbox{Count}(\mbox{A B})}{\mbox{Count}(\mbox{A})}$$

The full n-gram counts, $\mbox{ Count}(\mbox{A B})$, and the context n-gram counts, $\mbox{ Count}(\mbox{A})$, can be computed by counting n-grams in a **dataset**. For building a language model, this dataset is usually a collection of texts, also referred to as a **text corpus**.

**In this lab, you will**:

* Define your dataset, and break the sentences into individual tokens.
* Create n-grams from the tokenized tokens, and calculate counts of n-grams, $\mbox{ Count}(\mbox{A B})$.
* Estimate $P(\mbox{B} \mid \mbox{A})$ using the n-gram counts.
* Use the estimated $P(\mbox{B} \mid \mbox{A})$ distributions to generate new text based on your n-gram  language model.



## How to use Google Colaboratory (Colab)

Google Colaboratory (also known as Google Colab) is a platform that allows you to run Python code in your browser. The code is written in *cells* that are executed on a remote server.

To run a cell, hover over a cell and click on the `run` button to its left. The run button is the circle with the triangle (▶). Alternatively, you can also click on a cell and use the keyboard combination Ctrl+Return (or ⌘+Return if you are using a Mac).

To try this out, run the following cell. This should print today's day of the week below it.

In [1]:
from datetime import datetime
print(f"Today is {datetime.today():%A}.")

Today is Saturday.


Note that the *order in which you run the cells matters*. When you are working through a lab, make sure to always run *all* cells in order, otherwise the code might not work. If you take a break while working on a lab, Colab may disconnect you and in that case, you have to execute all cells again before  continuing your work. To make this easier, you can select the cell you are currently working on and then choose **Runtime** → **Run before**  from the menu above (or use the keyboard combination Ctrl/⌘ + F8). This will re-execute all cells before the current one.

In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

The code in this lab uses the [`random`](https://docs.python.org/3/library/random.html) package for sampling from probability distributions, the [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) and [`defaultdict`](https://docs.python.org/3/library/collections.html#collections.defaultdict) data types for counting n-grams, and the [`pandas`](https://pandas.pydata.org/) (`pd`) package for constructing data tables.

In [4]:
import sys
import os

sys.path.append(os.path.abspath('../..'))

# Packages used.
import random # For sampling from probability distributions.
from collections import Counter, defaultdict # For counting n-grams.

import textwrap # For automatically addding linebreaks to long texts.
import pandas as pd # For construction and visualizing tables.

# Custom functions for providing feedback on your solutions.
from ai_foundations.feedback.course_1 import ngrams

## Dataset loading and tokenization

Begin by loading the dataset that you will use to estimate the n-gram counts. For this purpose, you will process the  [AfricaGalore](https://storage.googleapis.com/dm-educational/assets/ai_foundations/africa_galore.json) dataset.

The Africa Galore dataset has been designed for this course and consists of synthetically generated paragraphs focusing on diverse aspects of African culture, history, and geography. It has been generated using Google's Gemini language model. Because it is synthetically created, the data is clean, free from the noise and inconsistencies that are often present in real-world datasets. At the same time, given its synthetic nature, the texts may not always be as natural as human-authored texts.

The dataset is specifically designed for educational purposes and the generation process has been guided to ensure that the content is concentrated around the topics relevant to the lab exercises you will be exploring. Its generation process was inspired by the [TinyStories project](https://arxiv.org/abs/2305.07759) [1].

Run the following cell to download the dataset.

In [6]:
africa_galore = pd.read_json(
    "../../assets/africa_galore.json"
)
dataset = africa_galore["description"]
print(f"The dataset consists of {dataset.shape[0]} paragraphs.")

The dataset consists of 232 paragraphs.


To get a sense of what these paragraphs look like, inspect the first ten paragraphs in the dataset. You may also want to skim the remainder of the dataset here: [Africa Galore dataset](https://storage.googleapis.com/dm-educational/assets/ai_foundations/africa_galore.json). When working with datasets, it is important to have a good sense of what is in the dataset, since datasets strongly influence the behavior of machine learning models.

In [7]:
for paragraph in dataset[:10]:
    # textwrap automatically adds linebreaks to make long texts more readable.
    formatted_paragraph = textwrap.fill(paragraph)
    print(f"{formatted_paragraph}\n")

The Lagos air was thick with humidity, but the energy in the club was
electric. The band launched into a hypnotic Afrobeat groove, the drums
pounding out a complex polyrhythm, the horns blaring a soaring melody,
and the bass laying down a deep, funky foundation. A woman named Imani
moved effortlessly to the music, her body swaying in time with the
rhythm. The music seemed to flow through her, a powerful current of
energy and joy. All around her, people were dancing, singing, and
clapping, caught up in the infectious rhythm. The music was more than
just entertainment; it was a celebration of life, a connection to
their shared heritage, a vibrant expression of the soul of Lagos.

The warm evening air in Accra was filled with the lilting melodies of
Highlife music. At a small bar tucked away on a side street, a band
played, the guitars weaving intricate patterns, the horns adding a
bright, joyful counterpoint. Kwame, a man with a wistful smile, sat at
a table nursing a beer, lost in the m

### Tokenization

Remember that an n-gram is a sequence of $n$ *words*. However, in its current form, the paragraphs are one long string. In order to split the dataset into n-grams and to count them so that you can use them to build a language model, you will have to split these sequences into individual words. This process is referred to as **tokenization**.

The simplest tokenizer is a **space tokenizer**. This tokenizer breaks sentences into individual words based on spaces, that is, the characters that are produced by pressing the space bar. For example, a space tokenizer would tokenize the sentence "Bimpe didn't buy the rice" into the list of words `["Bimpe", "didn't", "buy", "the", "rice"]`.

The cell below implements a space tokenizer in the `space_tokenize` function.

In [8]:
def space_tokenize(text: str) -> list[str]:
    """Splits a string into a list of words (tokens).

    Splits text on space.

    Args:
        text: The input text.

    Returns:
        A list of tokens. Returns empty list if text is empty or all spaces.
    """
    tokens = text.split(" ")
    return tokens

# Tokenize an example text with the `space_tokenize` function.
space_tokenize("Kanga, a colorful printed cloth is more than just a fabric.")

['Kanga,',
 'a',
 'colorful',
 'printed',
 'cloth',
 'is',
 'more',
 'than',
 'just',
 'a',
 'fabric.']

Note that the space tokenizer is quite naive. When you are tokenizing based only on spaces, you will observe that the punctuation marks are often considered part of the words. For example, tokenizing the sentence "Table mountain is tall."  will result in a different list of words than tokenizing "Table mountain is tall". The first sentence results in (`["Table", "mountain", "is", "tall."]`. The second sentence, which has the period missing from the end, will result in `["Table", "mountain", "is", "tall"]`). Since the units that tokenizers split sequences into are not always a word, they are usually referred to as **tokens**. In many cases, a token will be the same as a word but it may also be non-word strings, such as "tall." or "3/4".

You will learn about more sophisticated methods for tokenizing texts in later courses. For the purpose of this lab, the space tokenizer will be sufficient and you will use the implementation of the `space_tokenize` function that you observed in the previous cell.

To get an impression of what the tokenized data looks like, run the cell below to tokenize the first paragraph in the dataset.

In [9]:
space_tokenize(dataset[0])

['The',
 'Lagos',
 'air',
 'was',
 'thick',
 'with',
 'humidity,',
 'but',
 'the',
 'energy',
 'in',
 'the',
 'club',
 'was',
 'electric.',
 'The',
 'band',
 'launched',
 'into',
 'a',
 'hypnotic',
 'Afrobeat',
 'groove,',
 'the',
 'drums',
 'pounding',
 'out',
 'a',
 'complex',
 'polyrhythm,',
 'the',
 'horns',
 'blaring',
 'a',
 'soaring',
 'melody,',
 'and',
 'the',
 'bass',
 'laying',
 'down',
 'a',
 'deep,',
 'funky',
 'foundation.',
 'A',
 'woman',
 'named',
 'Imani',
 'moved',
 'effortlessly',
 'to',
 'the',
 'music,',
 'her',
 'body',
 'swaying',
 'in',
 'time',
 'with',
 'the',
 'rhythm.',
 'The',
 'music',
 'seemed',
 'to',
 'flow',
 'through',
 'her,',
 'a',
 'powerful',
 'current',
 'of',
 'energy',
 'and',
 'joy.',
 'All',
 'around',
 'her,',
 'people',
 'were',
 'dancing,',
 'singing,',
 'and',
 'clapping,',
 'caught',
 'up',
 'in',
 'the',
 'infectious',
 'rhythm.',
 'The',
 'music',
 'was',
 'more',
 'than',
 'just',
 'entertainment;',
 'it',
 'was',
 'a',
 'celebration

## Coding Activity 1: From lists of tokens to n-grams

The `space_tokenize` function returns a list of individual tokens. However, to compute the conditional probability of a token $\mbox{B}$ following a context $\mbox{A}$, $P(\mbox{B} \mid \mbox{A})$, you need to determine how often all n-grams and (n-1)-grams appear in your dataset.

For example, if you want to compute the probability of the token "is" following a bigram (2-gram) "Table Mountain", then you need to know the counts of the trigram (3-gram) "Table Mountain is" and the bigram "Table Mountain". More generally, to build an n-gram language model, you need to determine the counts of all n-grams and (n-1)-grams. As a first step towards obtaining these counts, you will write a function that turns a list of tokens into a list of n-grams.

------
> 💻 **Your task**:
>
> Complete the function `generate_ngrams(text: str, n: int)` below.
>
> This function should return a list of n-grams of length $n$ for a text. Each n-gram should be represented as a tuple of tokens. The function should therefore return a list of tuples of strings. You can use the [`tuple()`](https://www.w3schools.com/python/python_tuples.asp) function to convert a list of strings to a tuple of strings.
>
> Your function will first have to tokenize the text. You can use the `space_tokenize` function from above for this purpose. Second, the function needs to construct the list of n-grams for the text.
>
> For example, if the input to the function is `text = "Table Mountain is tall."` and `n = 2`, the function should return the following list of bigrams:
> ```
> [
>  ("Table", "Mountain"),
>  ("Mountain", "is"),
>  ("is", "tall.")
> ]
> ```
>
> Once you have finished your implementation, run the cell below to print the first ten unigrams, bigrams, and trigrams that appear in the dataset.
------

In [11]:
all_unigrams = []
all_bigrams = []
all_trigrams = []

def generate_ngrams(text: str, n: int) -> list[tuple[str]]:
    """Generates n-grams from a given text.

    Args:
        text: The input text string.
        n: The size of the n-grams (e.g., 2 for bigrams, 3 for trigrams).

    Returns:
        A list of n-grams, each represented as a list of tokens.
    """

    # Tokenize text.
    tokens = space_tokenize(text)

    if len(tokens) < n:
        return []

    # Construct the list of n-grams.
    ngrams = []

    for i in range(len(tokens) - n + 1):
        # 提取从位置i开始的n个连续词语
        ngram = tuple(tokens[i:i+n])
        ngrams.append(ngram)

    # Add your code here.

    return ngrams

for paragraph in dataset:
    # Calling `generate_ngrams` with n=1 constructs a list of unigrams.
    all_unigrams.extend(generate_ngrams(paragraph, n=1))
    # Calling `generate_ngrams` with n=2 constructs a list of bigrams (2-grams).
    all_bigrams.extend(generate_ngrams(paragraph, n=2))
    # Calling `generate_ngrams` with n=2 constructs a list of trigram (3-grams).
    all_trigrams.extend(generate_ngrams(paragraph, n=3))

print("First 10 Unigrams:", all_unigrams[:10])
print("First 10 Bigrams:", all_bigrams[:10])
print("First 10 Trigrams:", all_trigrams[:10])

First 10 Unigrams: [('The',), ('Lagos',), ('air',), ('was',), ('thick',), ('with',), ('humidity,',), ('but',), ('the',), ('energy',)]
First 10 Bigrams: [('The', 'Lagos'), ('Lagos', 'air'), ('air', 'was'), ('was', 'thick'), ('thick', 'with'), ('with', 'humidity,'), ('humidity,', 'but'), ('but', 'the'), ('the', 'energy'), ('energy', 'in')]
First 10 Trigrams: [('The', 'Lagos', 'air'), ('Lagos', 'air', 'was'), ('air', 'was', 'thick'), ('was', 'thick', 'with'), ('thick', 'with', 'humidity,'), ('with', 'humidity,', 'but'), ('humidity,', 'but', 'the'), ('but', 'the', 'energy'), ('the', 'energy', 'in'), ('energy', 'in', 'the')]


In [12]:
# @title Run this cell to test your implementation.
ngrams.test_generate_ngrams(generate_ngrams, space_tokenize)

✅ Nice! Your implementation looks correct.


The main reason for counting n-grams and using these counts to compute probabilities is that these probabilities can capture **patterns** in language. In this context, frequent n-grams are usually more interesting than very rare n-grams since they capture frequent co-occurrences of words (e.g., "Table Mountain" or "jollof rice").

Run the following cell to compute which n-grams appear most frequently in the dataset.
The output of the cell below is a list of tuples of the format `(ngram, number of occurrences)`. For example, the output shows you that the bigram `("is",  "a")` appears 144 times in the Africa Galore dataset.

In [13]:
# Use the Python Counter data type for computing the counts of all bigrams.
# See: https://docs.python.org/3/library/collections.html#collections.Counter
bigram_counts = Counter(all_bigrams)

# Print the ten most common bigrams.
print("Most common bigrams:")
for bigram, count in bigram_counts.most_common(10):
    print(f"  ({bigram}, {count})")

# Use the Python Counter data type for computing the counts of all trigrams.
trigram_counts = Counter(all_trigrams)

# Print the ten most common trigrams.
print("\n\nMost common trigrams:")
for trigram, count in trigram_counts.most_common(10):
    print(f"  ({trigram}, {count})")

Most common bigrams:
  (('is', 'a'), 144)
  (('of', 'the'), 100)
  (('and', 'the'), 69)
  (('in', 'the'), 61)
  (('with', 'a'), 60)
  (('in', 'a'), 55)
  (('and', 'a'), 50)
  (('to', 'the'), 42)
  (('was', 'a'), 39)
  (('It', 'is'), 33)


Most common trigrams:
  (('went', 'looking', 'for'), 32)
  (('a', 'symbol', 'of'), 18)
  (('was', 'hungry', 'so'), 18)
  (('The', 'result', 'is'), 17)
  (('looking', 'for', 'a'), 17)
  (('she', 'went', 'looking'), 16)
  (('he', 'went', 'looking'), 16)
  (('result', 'is', 'a'), 15)
  (('so', 'he', 'went'), 14)
  (('so', 'she', 'went'), 14)


## Coding Activity 2: Counting n-grams

In preparation for computing the probabilities, you require a function that returns the counts of n-grams.

------
> 💻 **Your task**:
>
> Complete the function `get_ngram_counts(dataset, n)` below.
>
> This function should return a dictionary of [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) objects where the keys are contexts of length n-1 tokens and the values are counters for the last token in the n-gram.
>
>For example, if the dataset consists of the two sentences "Table Mountain is tall." and "Table Mountain is beautiful." then the function called with `n = 3` should return:
>```
>{
>   "Table Mountain": Counter({"is": 2}),
>   "Mountain is": Counter({"tall": 1, "beautiful": 1})   
>}
>```
------

In [14]:
def get_ngram_counts(dataset: list[str], n: int) -> dict[str, Counter]:
    """Computes the n-gram counts from a dataset.

    This function takes a list of text strings (paragraphs or sentences) as
    input, constructs n-grams from each text, and creates a dictionary where:

    * Keys represent n-1 token long contexts `context`.
    * Values are a Counter object `counts` such that `counts[next_token]` is the
      count of `next_token` following `context`.

    Args:
        dataset: The list of text strings in the dataset.
        n: The size of the n-grams to generate (e.g., 2 for bigrams, 3 for
            trigrams).

    Returns:
        A dictionary where keys are (n-1)-token contexts and values are Counter
        objects storing the counts of each next token for that context.

    """

    # Define the dictionary as a defaultdict that is automatically initialized
    # with an empty Counter object. This allows you to access and set the value
    # of ngram_counts[context][next_token] without initializing
    # ngram_counts[context] or ngram_counts[context][next_token] first.
    # Reference
    # https://docs.python.org/3/library/collections.html#collections.Counter and
    # https://docs.python.org/3/library/collections.html#collections.defaultdict
    # for more information on how to use defaultdict and Counter types.
    ngram_counts = defaultdict(Counter)

    for paragraph in dataset:
        # 为每个段落生成n-gram
        for ngram in generate_ngrams(paragraph, n):
            # 提取上下文（所有词除最后一个）
            context = " ".join(ngram[:-1])
            # 提取下一个词（最后一个词）
            next_token = ngram[-1]
            # 增加计数
            ngram_counts[context][next_token] += 1

    return dict(ngram_counts)


# Example usage of the function.
example_data = [
    "This is an example sentence.",
    "Another example sentence.",
    "Split a sentence."
]
ngram_counts = get_ngram_counts(example_data, 2)

# Print the bigram counts dictionary for the dataset consisting of the
# three example sentences.
print("Bigram counts dictionary:\n")
print("{")
for context, counter in ngram_counts.items():
    print(f"  '{context}': {counter},")
print("}")

Bigram counts dictionary:

{
  'This': Counter({'is': 1}),
  'is': Counter({'an': 1}),
  'an': Counter({'example': 1}),
  'example': Counter({'sentence.': 2}),
  'Another': Counter({'example': 1}),
  'Split': Counter({'a': 1}),
  'a': Counter({'sentence.': 1}),
}


As you can see in the output, the count of the bigram "example sentence." is 2, which is shown in the entry for `"example"`.



In [15]:
# @title Run this cell to test your implementation.
ngrams.test_ngram_counts(get_ngram_counts, generate_ngrams)

✅ Nice! Your implementation looks correct.


### Exploring the n-gram counts in the Africa Galore dataset

You can now use the `get_ngram_counts` function to compute the bigram counts for all combinations of tokens in the Africa Galore dataset.

Run the following cell to print a table of bigram counts. This table shows the count of all bigrams where the first token in the bigram is shown at the beginning of each row and the second token is shown at top of each column.

In [16]:
bigram_counts = get_ngram_counts(dataset, n=2)

# Use the pandas library to display the counts in a table.
bigram_counts_matrix = {
    context: dict(counts) for context, counts in bigram_counts.items()
}
bigram_data_frame = pd.DataFrame.from_dict(
    bigram_counts_matrix, orient="index").fillna(0)

display(bigram_data_frame)

zero_count = (bigram_data_frame == 0).sum().sum()
print(
    f"Number of bigrams with a count of 0: {zero_count:,}"
    f" ({zero_count/bigram_data_frame.size * 100:.2f}%)"
)

,Lagos,band,music,warm,Highlife,bustling,Dakar,Mbalax,Kinshasa,Soukous,...,"kudu,","mph),",Ostriches,Antarctic,plumage,surface.,(Spheniscus,demersus).,breed,Bay
The,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
of,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the,0.0,1.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a,0.0,1.0,0.0,6.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
with,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
water's,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Penguin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
(Spheniscus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
penguins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Number of bigrams with a count of 0: 26,606,550 (99.95%)


As you can observe from the table, a lot of entries are 0. This is because the table shows (almost) every possible combination of all tokens that appear in the Africa Galore dataset. In total, there are $5,143 \times 5,176$ possible combinations but most of them (99.95%) never appear in the dataset. This **sparsity** is an important property to consider when building n-gram language models. For any context $\mbox{A}$, the probability $P(\mbox{B} \mid \mbox{A})$ will be 0 for most tokens $\mbox{B}$.




The sparsity increases even more as the length of the context increases. For an example of this, run the following cell, which computes the frequencies of all trigrams and displays them as a table:

In [17]:
trigram_counts = get_ngram_counts(dataset, n=3)

# Use the pandas library to display the counts in a table.
trigram_counts_matrix = {
    context: dict(counts) for context, counts in trigram_counts.items()
}
trigram_data_frame = pd.DataFrame.from_dict(
    trigram_counts_matrix, orient="index").fillna(0)

display(trigram_data_frame)

zero_count = (trigram_data_frame == 0).sum().sum()
print(
    f"Number of trigrams with a count of 0: {zero_count:,}"
    f" ({zero_count/trigram_data_frame.size * 100:.2f}%)"
)

,air,was,thick,thin,always,"quiet,",filled,alive,with,"humidity,",...,plumage,water's,surface.,penguin,(Spheniscus,demersus).,penguins,breed,Algoa,Bay
The Lagos,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
in the,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
and the,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
warm evening,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vegetables. The,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Penguin (Spheniscus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
demersus). These,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
These penguins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
down to,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Number of trigrams with a count of 0: 68,942,324 (99.98%)


The table above contains even more entries since it contains one entry for each combination of bigram (the columns) and token (the rows) that appears in the dataset. In the case of Africa Galore, this results in $13,411\times 5,142$ combinations. In this example, an even higher percentage of entries (99.98%) in the table are 0. Keep the concept of sparsity in mind as you use these counts to compute probabilities and use the probabilities to generate texts.



## Calculate P(B | A)

In the previous activities, you have laid all the groundwork for performing the last computation: computing the probability of a token $\mbox{B}$ following a context $\mbox{A}$.

$$P(\mbox{B} \mid \mbox{A}) = \frac{\mbox{Count}(\mbox{A B})}{\mbox{Count}(\mbox{A})}$$

Using the `get_ngram_counts` function above, you can compute both $\mbox{Count}(\mbox{A B})$ and $\mbox{Count}(\mbox{A})$. For example, if you wanted to estimate probabilities of a trigram model that uses a context of length 2, you could compute the counts in the numerator and the denominator for a `dataset` as:

```python
# Counts for the numerator.
trigram_counts = get_ngram_counts(dataset, n=3)
# Counts for the denominator.
bigram_counts = get_ngram_counts(dataset, n=2)
```

However, there is a small trick that computes the bigram counts directly from the trigram counts without calling `get_ngram_counts` a second time.

To observe how this works, consider the trigram counts for all trigrams that start with the bigram "a staple." You can access these using the dictionary `trigram_counts` that is defined above:

In [18]:
context = "a staple"
trigram_counts[context]

Counter({'in': 6,
         'dish': 2,
         'food': 1,
         'throughout': 1,
         'of': 1,
         'at': 1,
         'beverage': 1})

The counter in the output of the previous cell shows you that the dataset contains the following trigrams starting with "a staple":

* "a staple food" (1 time).
* "a staple in" (6 times).
* "a staple dish" (2 times).
* "a staple throughout" (1 time).
* "a staple of" (1 time).
* "a staple at" (1 time).
* "a staple beverage" (1 time).

The trick to get the bigram count of "a staple" is to sum the number of trigrams that start with "a staple." From the counter above, we can compute this total by summing $1+6+2+1+1+1+1 = 13$.

The following cell shows you how to do this automatically using the `sum()` function and the `values()` method of a counter.

In [19]:
context = "a staple"
# Compute the bigram count for "a staple" with sum().
bigram_count_a_staple = sum(trigram_counts[context].values())

print(
    'Bigram count of "a staple" computed indirectly from trigram counts: ',
    bigram_count_a_staple,
)

# Extract the bigram count for "a staple" from bigram_counts.
print('Bigram count of "a staple" computed directly: ',
      bigram_counts["a"]["staple"])

Bigram count of "a staple" computed indirectly from trigram counts:  13
Bigram count of "a staple" computed directly:  13


As the output of the cell above shows, both using the `sum()` function and computing the bigram counts indirectly results in the same number.

### Coding Activity 3: Computing the n-gram probabilities

------
> 💻 **Your task**:
>
> Complete the function `build_ngram_model(dataset, n)` below.
>
> This function should return a dictionary of dictionaries where the keys are contexts of length $n-1$ tokens and the values are a dictionary providing the probabilities of the next token given the context.
>
>For example, if the dataset consists of the two sentences "Table Mountain is tall." and "Table Mountain is beautiful." then the function called with `n = 3` should return:
>```
>{
>   "Table Mountain": {"is": 1.0},
>   "Mountain is": {"tall": 0.5, "beautiful": 0.5}   
>}
>```
------

In [20]:
def build_ngram_model(
    dataset: list[str],
    n: int
) -> dict[str, dict[str, float]]:
    """Builds an n-gram language model.

    This function takes a list of text strings (paragraphs or sentences) as
    input, generates n-grams from each text using the function get_ngram_counts
    and converts them into probabilities.  The resulting model is a dictionary,
    where keys are (n-1)-token contexts and values are dictionaries mapping
    possible next tokens to their conditional probabilities given the context.

    Args:
        dataset: A list of text strings representing the dataset.
        n: The size of the n-grams (e.g., 2 for a bigram model).

    Returns:
        A dictionary representing the n-gram language model, where keys are
        (n-1)-tokens contexts and values are dictionaries mapping possible next
        tokens to their conditional probabilities.
    """

    # A dictionary to store P(B | A).
    # ngram_model[context][token] should store P(token | context).
    ngram_model = {}

    # Use the ngram_counts as computed by the get_ngram_counts function.
    ngram_counts = get_ngram_counts(dataset, n)

    # Loop through the possible contexts. `context` is a string
    # and `next_tokens` is a dictionary mapping possible next tokens to their
    # counts of following `context`.
    for context, next_tokens in ngram_counts.items():

        # Compute Count(A) and P(B | A) here.
        # 计算上下文的總出现次数
        context_total_count = sum(next_tokens.values())
        
        # 为当前上下文创建概率字典
        ngram_model[context] = {}
        
        # 计算每个下一个token的条件概率
        for token, count in next_tokens.items():
            ngram_model[context][token] = count / context_total_count

    return ngram_model

# Test the method above by bulding a simple trigram model.
test_dataset = ["Table Mountain is tall.", "Table Mountain is beautiful."]
test_trigram_model = build_ngram_model(test_dataset, n=3)
test_trigram_model

{'Table Mountain': {'is': 1.0},
 'Mountain is': {'tall.': 0.5, 'beautiful.': 0.5}}

In [21]:
# @title Run this cell to test your implementation.
ngrams.test_build_ngram_model(build_ngram_model, get_ngram_counts)

✅ Nice! Your implementation looks correct.


After you have successfully implemented the method above, run the following cell to construct a trigram model that estimates the probabilities from the Africa Galore dataset.

In [22]:
trigram_model = build_ngram_model(dataset, n=3)

To gain an understanding of the patterns that the model learned, inspect a few probability distributions.

In [23]:
print(f"P(B | \"as it\") = {trigram_model['as it']}")

print(f"P(B | \"as they\") = {trigram_model['as they']}")

P(B | "as it") = {'is': 0.6666666666666666, 'receives': 0.3333333333333333}
P(B | "as they") = {'were': 1.0}


------
> 💭 **Reflection:**
>
> Do the probabilities that you estimated from the dataset make sense? Do they capture any patterns or rules of English that you know of?
------

As a final step in this part of the lab, look at the probability distribution for more contexts. Start with the context "The name."

In [24]:
context = "The name"
trigram_model[context]

{'means': 0.6666666666666666, "'Etosha'": 0.3333333333333333}

Now run the same code with a slightly altered context of "Their name."

In [26]:
context = "Their name"
trigram_model[context]

KeyError: 'Their name'

As you might observe, when you run the previous cell, this code results in an error. The reason for this is that the bigram "Their name" does not exist in the dataset. This means that it is not included in the trigram model that you built from the dataset and the dictionary that stores the probabilities does not contain an entry for this bigram, which is the cause of the `KeyError`.

This highlights one limitation of the n-gram language model: for some contexts it cannot generate continuations. While there exist extensions to n-gram models that make them more robust, they are generally limited in their ability to generate continuations for arbitrary contexts.


## Coding Activity 4: Using n-gram probabilities to sample next token

The purpose of counting n-grams and using them to estimate conditional probability distributions, as you did in the previous activities, was to be able to sample from the distributions to generate new texts. In this activity, you will now explore how you can sample the next token using an n-gram language model.

As a first step, consider again the code from the previous lab that used the `random.choices` function to sample a token from a list of candidate tokens, repeated in the next cell.

Recall that previously, you manually defined the possible next tokens and probabilities. During this activity, you will use the estimated probabilities from the n-gram model to define the possible next tokens and the associated probabilities.


Run the cell below multiple times to see different candidate words being picked:

In [27]:
# Define a list of tokens.
example_candidate_tokens = ["apple", "banana", "cherry"]

# Define corresponding probabilities for each fruit.
probabilities = [0.2, 0.5, 0.3]

# Sample one fruit based on the probabilities.
# The 'k=1' parameter instructs the function to return one item.
chosen_fruit = random.choices(
    example_candidate_tokens,
    weights=probabilities,
    k=1)[0]

print("Chosen fruit:", chosen_fruit)

Chosen fruit: apple


------
> 💻 **Your task:**
>
> Complete the following cell and use the probabilities in `trigram_model` to
> 1. Generate a list of candidate tokens for the context "looking for."
> 2. Extract the corresponding probabilities for each candidate token.
>
> Run the following cell multiple times to observe what tokens are being sampled.
------


In [28]:
context = "looking for"
candidate_tokens = []
candidate_tokens_probabilities = []

# Extract candidate tokens and associated probabilities from `trigram_model`.
# Add your code here.
if context in trigram_model:
    for token, probability in trigram_model[context].items():
        candidate_tokens.append(token)
        candidate_tokens_probabilities.append(probability)


print(f"Candidate tokens: {candidate_tokens}")
print(f"Candidate token probabilities: {candidate_tokens_probabilities}")

# Sample from the list of candidate tokens according to the
# associated probabilities.
next_token = random.choices(candidate_tokens,
                            weights=candidate_tokens_probabilities)[0]

print("\n\nSampled next token:")
print(context, next_token)

Candidate tokens: ['the', 'a', 'Banku', 'Tella,', 'Maafe,', 'Umqombothi,', 'sugarcane', 'crispy', 'warm', 'Doro', 'sambusa,', 'dodo,', 'Fura']
Candidate token probabilities: [0.125, 0.53125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125]


Sampled next token:
looking for a


In [29]:
# @title Run this to test your code, or get a hint.
ngrams.test_candidate_tokens(
    trigram_model, candidate_tokens, candidate_tokens_probabilities
)


✅ Nice! Your answer looks correct.


## Generating texts

You will now investigate the behavior of a function that can generate new texts for a given prompt using the probabilities of an n-gram model.

Text generation using an n-gram model is an iterative process where each newly generated token is added to the existing context. This forms the basis for predicting the next token.

Starting with an initial prompt text, the model uses the probability distribution derived from the n-gram counts to select the most likely next token. This again makes use of the `random.choices` function for picking the next token. Once this token has been generated, it is added to the context and the updated sequence is used to calculate the next probability distribution. This chain-like process continues until `num_tokens_to_generate` tokens have been generated.

The following `generate_next_n_tokens` function implements this iterative generation process:

In [30]:
def generate_next_n_tokens(
    n: int,
    ngram_model: dict[str, dict[str, float]],
    prompt: str,
    num_tokens_to_generate: int,
) -> str:
    """Generates `num_tokens_to_generate` tokens following a given prompt using
    an n-gram language model.

    This function takes an n-gram model and uses it to predict the most
    likely next token for the given prompt. The generation process
    continues iteratively, appending predicted tokens to the prompt until the
    desired number of tokens is generated or a context is
    encountered for which the model has no predictions.

    Args:
        n: The size of the n-grams to use (e.g., 2 for a bigram model).
        ngram_model: A dictionary representing the n-gram language model.
        prompt: The starting text prompt for generating the next tokens.
        num_tokens_to_generate: The number of words to generate following
            the prompt.

    Returns:
        A string containing the original prompt followed by the generated
        tokens. If no valid continuation is found for a given context, the
        function will return the text generated up to that point and print a
        message indicating that no continuation could be found.
    """

    # Split prompt into individual tokens.
    generated_words = space_tokenize(prompt)

    for _ in range(num_tokens_to_generate):
        # Get last (n-1) tokens as context.
        context = generated_words[-(n - 1):]
        context = " ".join(context)
        if context in ngram_model:
            # Sample next word based on probabilities.
            next_word = random.choices(
                list(ngram_model[context].keys()),
                weights=ngram_model[context].values()
            )[0]

            generated_words.append(next_word)
        else:
            print(
                "⚠️ No valid continuation found. Change the prompt or"
                " try sampling another continuation.\n"
            )
            break

    return " ".join(generated_words)

### Generating texts with a bigram model

First, run the following cell multiple times to generate new continuations using a bigram model whose probabilities were estimated from the Africa Galore dataset.


In [31]:
prompt = "Jide was hungry so she went looking for"

# Construct a bigram model using the Africa Galore dataset.
bigram_model = build_ngram_model(dataset, n=2)

n = 2  # Bigram.
num_tokens_to_generate = 10  # Generate next n words.
generate_next_n_tokens(
    n=n,
    ngram_model=bigram_model,
    prompt=prompt,
    num_tokens_to_generate=num_tokens_to_generate,
)

'Jide was hungry so she went looking for those who lived in Nigerian street pulsed with the largest'

### Generating texts with a trigram model

Next, run the following cell multiple times to generate new continuations using the trigram model that you built in the previous activities.


In [32]:
prompt = "Jide was hungry so she went looking for"

n = 3  # Trigram.
num_tokens_to_generate = 10  # Generate next n words.
generate_next_n_tokens(
    n=n,
    ngram_model=trigram_model,
    prompt=prompt,
    num_tokens_to_generate=num_tokens_to_generate,
)

'Jide was hungry so she went looking for a reptile, females exhibit strong maternal instincts, guarding their nests'

The different results when running the cell multiple times are because the n-gram model is a stochastic model that samples the next token from a probability distribution. More probable next words have a high probability of getting picked but are not guaranteed to be picked.

------
> 💭 **Reflection: Comparing the generations of bigram and trigram models**
>
> As you generate multiple continuations using both a bigram model and a trigram model, take note which continuations make more sense and tend to be grammatically correct. On average, does the bigram model or the trigram model produce more sensible continuations? Which model fails to produce a valid continuation more often?
------


## What happens when you increase the $n$ in n-grams?

While it intuitively seems that a larger context (greater $n$) would lead to better quality output by capturing more long-range dependencies in language,  it quickly runs into the problem of data sparsity since most n-grams will never be observed in the dataset.

When moving from bigrams (pairs of tokens) to trigrams (triplets of tokens), the number of possible combinations increases exponentially, and many of these triplets rarely, if ever, appear in the dataset. This means that while bigram models can cover a significant portion of common token pairs, the majority of potential token sequences in trigram and higher-order models are underrepresented. This makes it more challenging for the model to reliably predict the next token.

Consider a simple vocabulary of five tokens: "I", "love", "to", "eat", and "jollof". For bigrams, there are at most
$$5 \times 5 = 25 $$
possible combinations. In reality, however, your data might only include common pairs like "I love", "love to", "to eat", and "eat jollof". Now, when you move to trigrams (triplets of tokens), there are $$ 5 \times 5 \times 5 = 125$$ possible combinations. However, only a few of these, such as "I love to", "love to eat", and "to eat jollof", will actually appear in the data.

Even with massive datasets, many of the higher-order n-grams will never appear in the corpus. This results in many zero counts for the probabilities. As the n-gram order increases, the number of potential combinations grows exponentially. This often leads to many combinations being rare or absent in the data, which makes reliable estimation of probabilities more difficult.


## Summary

This is the end of the **Experiment with N-Gram Models** lab.

This lab provided a practical exploration of n-gram language models. Here are some key takeaways:

**1. Functionality:**

- You saw how n-gram models can be used to predict the next token in a sequence based on the preceding tokens (context).
- N-gram models are relatively simple to implement by estimating conditional probabilities from n-gram counts in a dataset. These probabilities can then be used to repeatedly sample the next token and generate new continuations.

**2. Data sparsity:**

- Data sparsity is a major challenge for n-gram models, especially with higher-order n-grams (trigrams or larger).
- This sparsity arises because many possible token combinations are rare or absent in real-world text data.
- You observed this in the dataset. The dimensions of the trigram matrix are significantly larger than those of the bigram matrix, resulting in more zero values.

**3. Randomness and text generation:**

- While the model assigns probabilities to different next tokens, the actual choice is stochastic (random), resulting in different outputs for multiple runs.
- While higher probabilities increase the chances of a token being picked, less frequent tokens can also be generated.

**4. Considerations for text generation:**

- The size of *n* can affect the quality of the generated text. Larger *n* might capture longer-range dependencies but can lead to data sparsity and repetitive outputs.
- The model is unable to generate text following an n-gram that is not present in the dataset.

In the next activity, you will reflect on some of the limitations of n-gram models and go on to compare them with more advanced models.

## Solutions

The following cells provide reference solutions to the coding activities above. If you really get stuck after trying to solve the activities yourself, you may want to consult these solutions.

However, we recommend that you *only* look at the solutions after you have tried to solve the activities above *multiple times*. The best way to learn challenging concepts in computer science and artificial intelligence is to debug your code piece-by-piece until it works rather than copying existing solutions.

If you feel stuck, you may want to first try to debug your code, for example, by adding additional print statements to see what your code is doing at every step. This will provide you with a much deeper understanding of the code and the materials. It will also provide you with practice on how to solve challenging coding problems beyond this course.

To view the solutions for an activity, click on the arrow to the left of the activity name. If you consult the solutions, do not copy and paste them into the cells above. Instead, look at them and then type them manually into the cell. This will help you understand where you went wrong.

### Coding Activity 1

In [ ]:
# This is a complete implementation of `generate_ngrams`.
def generate_ngrams(text: str, n: int) -> list[tuple[str]]:
    """Generates n-grams from a given text.

    Args:
        text: The input text string.
        n: The size of the n-grams (e.g., 2 for bigrams, 3 for trigrams).

    Returns:
        A list of n-grams, each represented as a list of tokens.
    """

    # Tokenize text.
    tokens = space_tokenize(text)

    # Construct the list of n-grams.
    ngrams = []

    num_of_tokens = len(tokens)

    # The last n-gram will be tokens[num_of_tokens - n + 1: num_of_tokens + 1].
    for i in range(0, num_of_tokens - n + 1):
        ngrams.append(tuple(tokens[i:i+n]))

    return ngrams

### Coding Activity 2

In [ ]:
# This is a complete implementation of get_ngram_counts.
def get_ngram_counts(dataset: list[str], n: int) -> dict[str, Counter]:
    """Computes the n-gram counts from a dataset.

    This function takes a list of text strings (paragraphs or sentences) as
    input, constructs n-grams from each text, and creates a dictionary where:

    * Keys represent n-1 token long contexts `context`.
    * Values are a Counter object `counts` such that `counts[next_token]` is the
    * count of `next_token` following `context`.

    Args:
        dataset: The list of text strings in the dataset.
        n: The size of the n-grams to generate (e.g., 2 for bigrams, 3 for
            trigrams).

    Returns:
        A dictionary where keys are (n-1)-token contexts and values are Counter
        objects storing the counts of each next token for that context.

    """

    # Define the dictionary as a defaultdict that is automatically initialized
    # with an empty Counter object. This allows you to access and set the value
    # of ngram_counts[context][next_token] without initializing
    # ngram_counts[context] or ngram_counts[context][next_token] first.
    # See
    # https://docs.python.org/3/library/collections.html#collections.Counter and
    # https://docs.python.org/3/library/collections.html#collections.defaultdict
    # for more information on how to use defaultdict and Counter types.
    ngram_counts = defaultdict(Counter)

    # Loop through all paragraphs.
    for paragraph in dataset:
        # Loop through all n-grams for the paragraph.
        for ngram in generate_ngrams(paragraph, n):
            # Extract the context. This will be all but the last token.
            context = " ".join(ngram[:-1])
            # Extract the next token. This will be the last token of the n-gram.
            next_token = ngram[-1]
            # Increment the counter for the context - next_token pair by 1.
            ngram_counts[context][next_token] += 1

    return dict(ngram_counts)

### Coding Activity 3


In [ ]:
# Complete implemenation of build_ngram_model.
def build_ngram_model(dataset: list[str], n: int) -> dict[str, dict[str, float]]:
    """Builds an n-gram language model.

    This function takes a list of text strings (paragraphs or sentences) as
    input, generates n-grams from each text using the function get_ngram_counts
    and converts them into probabilities.  The resulting model is a dictionary,
    where keys are (n-1)-token contexts and values are dictionaries mapping
    possible next tokens to their conditional probabilities given the context.

    Args:
        dataset: A list of text strings representing the dataset.
        n: The size of the n-grams (e.g., 2 for a bigram model).

    Returns:
        A dictionary representing the n-gram language model, where keys are
        (n-1)-tokens contexts and values are dictionaries mapping possible next
        tokens to their conditional probabilities.
    """
    # A dictionary to store P(B | A).
    # ngram_model[context][token] should store P(token | context).
    ngram_model = {}

    # Use the ngram_counts as computed by the get_ngram_counts function.
    ngram_counts = get_ngram_counts(dataset, n)


    # Loop through the possible contexts. `context` is a string
    # and `next_tokens` is a dictionary mapping possible next tokens to their
    # counts of following `context`.
    for context, next_tokens in ngram_counts.items():

        # Compute Count(A) and P(B | A ) here.
        context_total_count = sum(next_tokens.values())
        ngram_model[context] = {}
        for token, count in next_tokens.items():
            ngram_model[context][token] = count / context_total_count

    return ngram_model

### Coding Activity 4

In [ ]:
# Include this code for extracting the candidate tokens and
# candidate tokens probabilities.

# Extract candidate tokens and associated probabilities from `trigram_model`.
for token, prob in trigram_model[context].items():
    candidate_tokens.append(token)
    candidate_tokens_probabilities.append(prob)


## References

[1] Ronen Eldan and Yuanzhi Li. 2023. Tiny Stories: How Small Can Language Models Be and Still Speak Coherent English. arXiv:2305.07759. Retrieved from [https://arxiv.org/pdf/2305.07759](https://arxiv.org/pdf/2305.07759).
